In [194]:
from datetime import timedelta
import pandas as pd
import numpy as np
import scipy.stats as ss
import re
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

In [ ]:
# read the file
train = pd.read_csv('/home/f/DS/Compete/L&T_Fin_Hack/train_aox2Jxw/train.csv')
test = pd.read_csv('/home/f/DS/Compete/L&T_Fin_Hack/test_bqCt9Pv.csv')

In [162]:
# comibne train and test data
train['source'] = 'train'
test['source'] = 'test'
df = pd.concat([train, test], ignore_index=True, sort=False)

In [163]:
df.head()

,UniqueID,disbursed_amount,asset_cost,ltv,branch_id,supplier_id,manufacturer_id,Current_pincode_ID,Date.of.Birth,Employment.Type,...,SEC.DISBURSED.AMOUNT,PRIMARY.INSTAL.AMT,SEC.INSTAL.AMT,NEW.ACCTS.IN.LAST.SIX.MONTHS,DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS,AVERAGE.ACCT.AGE,CREDIT.HISTORY.LENGTH,NO.OF_INQUIRIES,loan_default,source
0,420825,50578,58400,89.55,67,22807,45,1441,01-01-84,Salaried,...,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0.0,train
1,537409,47145,65550,73.23,67,22807,45,1502,31-07-85,Self employed,...,0,1991,0,0,1,1yrs 11mon,1yrs 11mon,0,1.0,train
2,417566,53278,61360,89.63,67,22807,45,1497,24-08-85,Self employed,...,0,0,0,0,0,0yrs 0mon,0yrs 0mon,0,0.0,train
3,624493,57513,66113,88.48,67,22807,45,1501,30-12-93,Self employed,...,0,31,0,0,0,0yrs 8mon,1yrs 3mon,1,1.0,train
4,539055,52378,60300,88.39,67,22807,45,1495,09-12-77,Self employed,...,0,0,0,0,0,0yrs 0mon,0yrs 0mon,1,1.0,train


In [164]:
def count_missing(df):
    missing_df = df.isnull().sum().reset_index()
    missing_df.columns = ['column_name', 'missing_count']
    missing_df = missing_df[missing_df['missing_count']>0]
    missing_df['missing_perc'] = missing_df['missing_count']/df.shape[0]*100
    missing_df = missing_df.sort_values(by='missing_count')
    return missing_df

count_missing(df)

,column_name,missing_count,missing_perc
9,Employment.Type,11104,3.213465
40,loan_default,112392,32.525916


In [165]:
count_missing(train)

,column_name,missing_count,missing_perc
9,Employment.Type,7661,3.285811


In [166]:
drop_cols = ['UniqueID']

In [167]:
df['disbursed__asset'] = df['asset_cost'] - df['disbursed_amount']

In [168]:
# age of applicant on disbursal date
df['DisbursalDate'] = pd.to_datetime(df['DisbursalDate'])
df['Date.of.Birth'] = pd.to_datetime(df['Date.of.Birth'])

df['DisbursalDate_year'] = df['DisbursalDate'].dt.year

In [169]:
df['DisbursalDate_quarter'] = df['DisbursalDate'].dt.quarter

In [170]:
df['DisbursalDate_weekday'] = df['DisbursalDate'].dt.weekday

In [171]:
df['age_in_days'] = df['DisbursalDate'] - df['Date.of.Birth']
df['age_in_years'] = df["age_in_days"] / timedelta(days=365)
df['age_in_years'] = df['age_in_years'].astype('float')

In [172]:
drop_cols = drop_cols + ['DisbursalDate', 'Date.of.Birth', 'DisbursalDate_year', 'age_in_days']

In [173]:
drop_cols.append('MobileNo_Avl_Flag')

In [174]:
df['PERFORM_CNS.SCORE.DESCRIPTION'] = df['PERFORM_CNS.SCORE.DESCRIPTION'].map({'A-Very Low Risk' : 'very_low',
                                                                               'B-Very Low Risk' : 'very_low',
                                                                               'C-Very Low Risk' : 'very_low',
                                                                               'D-Very Low Risk' : 'very_low',
                                                                               'E-Low Risk'      : 'low',
                                                                               'F-Low Risk'      : 'low',
                                                                               'G-Low Risk'      : 'low',
                                                                               'H-Medium Risk'   : 'medium',
                                                                               'I-Medium Risk'   : 'medium',
                                                                               'J-High Risk'     : 'high',
                                                                               'K-High Risk'     : 'high',
                                                                               'L-Very High Risk': 'very_high',
                                                                               'M-Very High Risk': 'very_high',
                                                                               'Not Scored: Sufficient History Not Available'           : 'not_scored',
                                                                               'Not Scored: Not Enough Info available on the customer'  : 'not_scored',
                                                                               'Not Scored: No Activity seen on the customer (Inactive)': 'not_scored',
                                                                               'Not Scored: No Updates available in last 36 months'     : 'not_scored',
                                                                               'Not Scored: Only a Guarantor'                           : 'not_scored',
                                                                               'Not Scored: More than 50 active Accounts found'         : 'not_scored',
                                                                               'No Bureau History Available'                            : 'not_scored'
                                                                              })

In [175]:
df['total.NO.OF.ACCTS'] = df['PRI.NO.OF.ACCTS'] + df['SEC.NO.OF.ACCTS']
df['total.ACTIVE.ACCTS'] = df['PRI.ACTIVE.ACCTS'] + df['SEC.ACTIVE.ACCTS']
df['total.OVERDUE.ACCTS'] = df['PRI.OVERDUE.ACCTS'] + df['SEC.OVERDUE.ACCTS']
df['total.CURRENT.BALANCE'] = df['PRI.CURRENT.BALANCE'] + df['SEC.CURRENT.BALANCE']
df['total.SANCTIONED.AMOUNT'] = df['PRI.SANCTIONED.AMOUNT'] + df['SEC.SANCTIONED.AMOUNT']
df['total.DISBURSED.AMOUNT'] = df['PRI.DISBURSED.AMOUNT'] + df['SEC.DISBURSED.AMOUNT']
df['total.INSTAL.AMT'] = df['PRIMARY.INSTAL.AMT'] + df['SEC.INSTAL.AMT']

In [176]:
df['total.DEACT.ACCTS'] = df['total.NO.OF.ACCTS'] + df['total.ACTIVE.ACCTS']

In [177]:
df['NON_DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS'] = df['NEW.ACCTS.IN.LAST.SIX.MONTHS'] + df['DELINQUENT.ACCTS.IN.LAST.SIX.MONTHS']

In [178]:
df['total.SANC_diff_DISB'] = df['total.SANCTIONED.AMOUNT'] - df['total.DISBURSED.AMOUNT']

In [179]:
df['total_flag'] = df['Driving_flag'] + df['Aadhar_flag'] + df['PAN_flag'] + df['Passport_flag'] + df['VoterID_flag']

In [180]:
def to_months(text):
    s = re.findall(r'\d+', text)    
    months = int(s[0])*12 + int(s[1])
    return months
to_months('2yrs 1mon')

25

In [181]:
df['CREDIT.HISTORY.LENGTH'] = df['CREDIT.HISTORY.LENGTH'].apply(to_months).astype('float')
df['AVERAGE.ACCT.AGE'] = df['AVERAGE.ACCT.AGE'].apply(to_months).astype('float')

In [182]:
df['Current_pincode_ID'] = df['Current_pincode_ID'].astype(np.object)
df['Employee_code_ID'] = df['Employee_code_ID'].astype(np.object)
df['State_ID'] = df['State_ID'].astype(np.object)
df['branch_id'] = df['branch_id'].astype(np.object)
df['manufacturer_id'] = df['manufacturer_id'].astype(np.object)
df['supplier_id'] = df['supplier_id'].astype(np.object)

In [189]:
data_train = df.loc[df.source=='train']
data_test = df.loc[df.source=='test']

target = 'loan_default'
predictors = [x for x in data_train.columns if x not in drop_cols+[target]+['source']]
dummify = ['DisbursalDate_quarter', 'DisbursalDate_weekday', 'Employment.Type', 'PERFORM_CNS.SCORE.DESCRIPTION']

X = data_train[predictors]
X = pd.get_dummies(X, columns=dummify)
y = data_train[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [200]:
params = {'booster' : "gbtree", 'max_depth':5, 'eta':0.04, 'silent':1, 'objective':'binary:logistic', 'colsample_bytree':0.6, 'min_child_weight':2, 'gamma':0.6, \
         'subsample':0.9, 'colsample_bytree':0.6, 'scale_pos_weight':5, 'eval_metric':'auc', 'lambda':0.3, 'max_delta_step':3, 'max_bin':400, 'random_state':37}
dtrain = xgb.DMatrix(X_train.values, label=y_train.values)
dtest = xgb.DMatrix(X_test.values, label=y_test.values)

watchlist = [(dtrain, 'train'), (dtest, 'valid')]
nbr = 300
esr = 10
model = xgb.train(params, dtrain, nbr, watchlist, maximize=False, early_stopping_rounds = esr, verbose_eval=20)

[0]	train-auc:0.6144	valid-auc:0.609122
Multiple eval metrics have been passed: 'valid-auc' will be used for early stopping.

Will train until valid-auc hasn't improved in 10 rounds.
[20]	train-auc:0.647391	valid-auc:0.641898
[40]	train-auc:0.655804	valid-auc:0.648382
[60]	train-auc:0.662306	valid-auc:0.65245
[80]	train-auc:0.667512	valid-auc:0.655138
[100]	train-auc:0.672457	valid-auc:0.657611
[120]	train-auc:0.676201	valid-auc:0.658993
[140]	train-auc:0.679476	valid-auc:0.660432
[160]	train-auc:0.682383	valid-auc:0.661457
[180]	train-auc:0.685107	valid-auc:0.66233
[200]	train-auc:0.687587	valid-auc:0.66289
[220]	train-auc:0.689883	valid-auc:0.663462
[240]	train-auc:0.692156	valid-auc:0.664154
[260]	train-auc:0.694082	valid-auc:0.66481
[280]	train-auc:0.696226	valid-auc:0.665387
[299]	train-auc:0.697958	valid-auc:0.665664


In [201]:
# prediction on test data
X_TEST = data_test[predictors]
X_TEST = pd.get_dummies(X_TEST, columns=dummify)
Y_TEST = data_test[target]
x_test = xgb.DMatrix(X_TEST.values)
pred_test = model.predict(x_test)

In [208]:
submission = pd.DataFrame({"UniqueID": test["UniqueID"], "loan_default": pred_test})
submission.to_csv('/home/f/DS/Compete/L&T_Fin_Hack/submission.csv', index=False)